In [1]:
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
import cv2
import tensorflow as tf
import pandas as pd
import keras
from keras import layers, regularizers
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-05-01 00:27:31.554486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 00:27:31.554624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 00:27:31.680765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
Shoulder_Path = "/kaggle/input/xception-shoulder/keras/xception-shoulder/3/MobilenetV2_shoulder.keras"

In [3]:
#Functions Declarations 
# Define required Preprocessing Funcs
def Clahe_Enhance(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    x, y, w, h = cv2.boundingRect(thresh)
    x, y, w, h = x, y, w + 20, h + 20
    img = img[y:y+h, x:x+w]
    gray_crop = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_crop)
    return clahe_img

def Normalize_IMG(img):
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = img.astype('float32') / 255.0
    return img

# For Kaggle 
def Copy_Model(source_path,name):
    destination_path = f"/kaggle/working/{name}.keras"
    shutil.copyfile(source_path, destination_path)
    return destination_path

def load_image_wrist(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    return img

# Models Functions 
# Shoulder Image prediction

def BoneType_Predict(img):
    #categories list
    categories_parts = ["Elbow","Finger", "Hand", "Shoulder","Wrist"]
    img_resized = cv2.resize(img, (224, 224))
    img_array = image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img_array = preprocess_input(img_array)# Preprocess the image using ResNet50 preprocessing function
    prediction = np.argmax(BoneType_Model.predict(preprocessed_img_array), axis=1)
    prediction_str = categories_parts[prediction.item()]
    return prediction_str

def Shoulder_Predict(img):
    img = Clahe_Enhance(img)
    img = Normalize_IMG(img)
    img = np.expand_dims(img, axis=0)
    prediction = Shoulder_Model.predict(img)
    print(prediction)
    binary_prediction = "Fractured" if prediction > 0.5 else "Non-Fractured"
    return binary_prediction

def Finger_Predict(img):
    img_resized = cv2.resize(img, (224, 224))
    img_array = image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img_array = keras.applications.resnet50.preprocess_input(img_array)
    prediction = Finger_Model.predict(preprocessed_img_array)
    binary_prediction = "Fractured" if prediction > 0.5 else "Non-Fractured"
    return binary_prediction

def Hand_Predict(img):
    img_resized = cv2.resize(img, (224, 224))
    img_array = image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    prediction = Hand_Model.predict(img_array)
    binary_prediction = "Fractured" if prediction > 0.72 else "Non-Fractured"
    return binary_prediction

def Elbow_Predict(img):
    img_resized = cv2.resize(img, (224, 224))
    img_array = image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img_array = keras.applications.resnet50.preprocess_input(img_array)
    prediction = Elbow_Model.predict(preprocessed_img_array)
    binary_prediction = "Fractured" if prediction > 0.5 else "Non-Fractured"
    return binary_prediction

def Wrist_Predict(img):
    img = np.expand_dims(img, 0)
    prediction = Wrist_Model.predict(img, verbose = 0)
    binary_prediction = "Fractured" if prediction > 0.5 else "Non-Fractured"
    return binary_prediction

def predict_fracture(img):
    Predicted_bonetype = BoneType_Predict(img)
    print(f"Predicted bone-type is {Predicted_bonetype}")
    if Predicted_bonetype == "Shoulder": 
        final_p = Shoulder_Predict(img)
    elif Predicted_bonetype == "Finger":
        final_p = Finger_Predict(img)
    elif Predicted_bonetype == "Wrist":
        final_p = Wrist_Predict(img)
    elif Predicted_bonetype == "Hand":
        final_p = Hand_Predict(img)
    elif Predicted_bonetype == "Elbow":
        final_p = Elbow_Predict(img)
    return final_p


In [4]:
# Source path of the models
#BoneType_Model_Path = Copy_Model(BoneType_Path,"BoneType")
Shoulder_Model_Path = Copy_Model(Shoulder_Path,"Shoulder")
#Finger_Model_Path = Copy_Model(Finger_Path,"Finger")
# Hand_Model_Path = Copy_Model(Hand_Path,"Hand")
#Elbow_Model_Path = Copy_Model(Elbow_Path,"Elbow")
#Wrist_Model_Path = Copy_Model(Wrist_Path,"Wrist")

In [5]:
# Models Loading
#BoneType_Model = load_model(BoneType_Model_Path)
Shoulder_Model = load_model(Shoulder_Model_Path)
#Finger_Model = load_model(Finger_Model_Path)
# Hand_Model = load_model(Hand_Model_Path)
#shutil.copyfile('/kaggle/input/alexnettrial/tensorflow2/alex2/1/alexNet.h5', '/kaggle/working/alexNet.h5')
#Hand_Model = model = tf.keras.models.load_model('/kaggle/working/alexNet.h5')
#Elbow_Model = load_model(Elbow_Model_Path)
#Wrist_Model = load_model(Wrist_Model_Path)

In [6]:
prefix = '/kaggle/input/better-mura/MURA-v1.1/'
valid_data = pd.read_csv('/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/valid.csv')
valid_data = valid_data[valid_data["BodyPart"].str.contains(r"XR_SHOULDER")]
# valid_data = valid_data[valid_data["BodyPart"].str.contains(r"XR_FINGER")]
# valid_data = valid_data[valid_data["BodyPart"].str.contains(r"XR_HAND")]
# valid_data = valid_data[valid_data["BodyPart"].str.contains(r"XR_ELBOW")]
# valid_data = valid_data[valid_data["BodyPart"].str.contains(r"XR_WRIST")]

# Loading all the data except Humerus and Forearm
#valid_data = valid_data[~valid_data["BodyPart"].str.contains(r"XR_HUMERUS|XR_FOREARM")]
    
paths = prefix + valid_data['path'].astype(str)
labels = valid_data['label']
labels = pd.Series(labels, name='Label')
labels = labels.astype(str)
bodyparts = valid_data['BodyPart'].astype(str)

#v_images = pd.concat([paths, labels, bodyparts], axis=1)
v_images = pd.concat([paths, labels], axis=1)

In [7]:
##### Test Fracture Model Accuracy Individually #####
fracturedcounter = 0
normalcounter = 0
prefracturedcounter = 0
prenormalcounter = 0
true_positive = 0
false_positive = 0
false_negative = 0
true_negative = 0

for index, row in v_images.iterrows():
    image_path = row["path"]
    if row["Label"] == '1':
        label = "Fractured"
        fracturedcounter += 1
    else:
        label = "Non-Fractured"
        normalcounter += 1
        
    img = cv2.imread(image_path)
    #img = load_image_wrist(image_path)
    
    ## PUT YOUR FUNCTION HERE ##
    prediction = Shoulder_Predict(img)
    
    if prediction == "Fractured":
        prefracturedcounter += 1
    else:
        prenormalcounter += 1
    print("Real Value: " + label)
    print("Status: " + prediction)
    if label == prediction:
        if label == "Fractured":
            true_positive += 1
        else:
            true_negative += 1
    else:
        if label == "Fractured":
            false_negative += 1
        else:
            false_positive += 1

accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Fractured Count: " + str(fracturedcounter) + " Normal Count: " + str(normalcounter))
print("preFractured Count: " + str(prefracturedcounter) + " preNormal Count: " + str(prenormalcounter))
print("Accuracy: ", accuracy*100)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1 Score: ", f1_score*100)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
[[0.99999416]]
Real Value: Fractured
Status: Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.99995077]]
Real Value: Fractured
Status: Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


I0000 00:00:1714523278.216549      65 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[[0.99996984]]
Real Value: Fractured
Status: Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.24220194]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.00191209]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.14827904]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.0030924]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.00075474]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.00079655]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.20972905]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.8885823]]
Real Value: Fractured
Status: Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.43836606]]
Real Value: Fractured
Status: Non-Fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.9985475]]
Real Value: Fractured
Stat